In [14]:
"""
https://github.com/xingyizhou/ExtremeNet
https://github.com/gladcolor/Mp_clip/blob/0120641a30/czhUtils.py
https://github.com/gladcolor/JupytetNoteBooks/blob/master/COCO_drawing.ipynb


https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
"""


from skimage import measure
import numpy as np
import json
import os
import datetime
from shapely.geometry import Polygon, Point

In [12]:
INFO = {
    "description": "image detection",
    "url": "",
    "version": "0.1.0",
    "year": 2020,
    "contributor": "njit",
    "date_created": datetime.datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "Attribution-NonCommercial-ShareAlike License",
        "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
    }
]

CATEGORIES = [
    {
        'id': 1,
        'name': 'doorway', # 门洞
        'supercategory': '',
    },
    {
        'id': 2,
        'name': 'interface', # 界面
        'supercategory': '',
    },
        {
        'id': 3,
        'name': 'window', # 窗口
        'supercategory': '',
    }
]

label_dic = {
    '门洞': 1,
    '界面': 2,
    '窗口': 3
}

In [41]:
image_id = 1
annotation_id = 1

unique_images = []

def process(input_path, output_filename):
    
    images = []
    annotations = []
    
    global image_id, annotation_id
    
    list_dirs = os.walk(input_path) 
    file_list = []
    for root, dirs, files in list_dirs: 
        for f in files: 
            if f[-5:] == '.json':
                full_file_name = os.path.join(root, f)
                file_list.append(full_file_name)
    
    for input_filename in file_list:
        with open(input_filename, 'r') as f:
            json_str = f.read()
            json_obj = json.loads(json_str)

        date_captured=datetime.datetime.utcnow().isoformat(' ')
        print(input_filename)

        for img_name in json_obj.keys():
            if img_name in unique_images:
                continue
            unique_images.append(img_name)
#             print(img_name)

            coco_url = ''
            flickr_url = ''
            license_id = 1
            width = 800
            height = 600
            images.append({
                "id": image_id,
                "file_name": img_name,
                "width": width,
                "height": height,
                "date_captured": date_captured,
                "license": license_id,
                "coco_url": coco_url,
                "flickr_url": flickr_url
            })

            regions = json_obj[img_name]['regions']
            for region_id in regions.keys():
                region = regions[region_id]

                label = region['region_attributes']['label']
                lab_id = label_dic[label]

                region_type = region['shape_attributes']['name']
                if region_type == 'polygon':
                    segmentation = []
                    all_points_x = region['shape_attributes']['all_points_x']
                    all_points_y = region['shape_attributes']['all_points_y']
                    pointList = []
                    for i in range(len(all_points_x)):
                        segmentation.append(all_points_x[i])
                        segmentation.append(all_points_y[i])
                        pointList.append( Point(all_points_x[i], all_points_y[i]) )

                    polygon = Polygon([[p.x, p.y] for p in pointList])
                    bbx =[0 if i < 0 else int(i) for i in list(polygon.bounds)]
                    bbox = [bbx[0], bbx[1], bbx[2]-bbx[0], bbx[3]-bbx[1]]

                    annotations.append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": lab_id,
                        "iscrowd": 0,
                        "area": polygon.area, # number of pixels shaded
                        "bbox": bbox, # [x,y,width,height]
                        "segmentation": [segmentation],
                    })

                    annotation_id += 1
                else:
                    print(" === New region_type:", region_type)        

            image_id += 1
    
    res = {
        "info": INFO, 
        "images": images, 
        "annotations": annotations, 
        "licenses": LICENSES,
        "categories": CATEGORIES
    }

    str_res = json.dumps(res)
    with open(output_filename, "w") as f:
        f.write(str_res)
        print(output_filename + " done")
        

In [42]:
process('input/train', 'train.json')
process('input/test', 'test.json')

input/train/1601-1800/labels_my-project-name_20200714072443.json
input/train/1601-1800/labels_my-project-name_20200714073213.json
input/train/1601-1800/labels_my-project-name_20200714072642.json
input/train/1601-1800/labels_my-project-name_20200714083342.json
input/train/1601-1800/labels_my-project-name_20200714065538.json
input/train/1601-1800/labels_my-project-name_20200714064515.json
input/train/1601-1800/labels_my-project-name_20200714090939.json
input/train/1601-1800/labels_my-project-name_20200714075332.json
input/train/1601-1800/labels_my-project-name_20200714084632.json
input/train/1601-1800/labels_my-project-name_20200714070425.json
input/train/1601-1800/labels_my-project-name_20200714073024.json
input/train/1601-1800/labels_my-project-name_20200714090722.json
input/train/1601-1800/labels_my-project-name_20200714084424.json
input/train/1601-1800/labels_my-project-name_20200714074324.json
input/train/1601-1800/labels_my-project-name_20200714075149.json
input/train/1601-1800/lab

input/test/2201-2500/labels_my-project-name_20200723061853.json
input/test/2201-2500/labels_my-project-name_20200723071629.json
input/test/2201-2500/labels_my-project-name_20200723073614.json
input/test/2201-2500/labels_my-project-name_20200723071042.json
input/test/2201-2500/labels_my-project-name_20200723084917.json
input/test/2201-2500/labels_my-project-name_20200723073511.json
input/test/2201-2500/labels_my-project-name_20200723054635.json
input/test/2201-2500/labels_my-project-name_20200723060556.json
input/test/2201-2500/labels_my-project-name_20200723085453.json
input/test/2201-2500/labels_my-project-name_20200723084429.json
input/test/2201-2500/labels_my-project-name_20200723070833.json
input/test/2201-2500/labels_my-project-name_20200723074827.json
input/test/2201-2500/labels_my-project-name_20200723075109.json
input/test/2201-2500/labels_my-project-name_20200723084809.json
input/test/2201-2500/labels_my-project-name_20200723072818.json
input/test/2201-2500/labels_my-project-n

In [43]:
unique_images

['001603.jpg',
 '001605.jpg',
 '001608.jpg',
 '001609.jpg',
 '001614.jpg',
 '001615.jpg',
 '001618.jpg',
 '001619.jpg',
 '001626.jpg',
 '001627.jpg',
 '001628.jpg',
 '001632.jpg',
 '001633.jpg',
 '001635.jpg',
 '001638.jpg',
 '001641.jpg',
 '001644.jpg',
 '001648.jpg',
 '001654.jpg',
 '001655.jpg',
 '001657.jpg',
 '001658.jpg',
 '001660.jpg',
 '001663.jpg',
 '001664.jpg',
 '001666.jpg',
 '001671.jpg',
 '001672.jpg',
 '001674.jpg',
 '001675.jpg',
 '001676.jpg',
 '001678.jpg',
 '001680.jpg',
 '001681.jpg',
 '001682.jpg',
 '001691.jpg',
 '001692.jpg',
 '001697.jpg',
 '001699.jpg',
 '001701.jpg',
 '001703.jpg',
 '001706.jpg',
 '001708.jpg',
 '001709.jpg',
 '001713.jpg',
 '001717.jpg',
 '001723.jpg',
 '001727.jpg',
 '001730.jpg',
 '001734.jpg',
 '001735.jpg',
 '001736.jpg',
 '001743.jpg',
 '001747.jpg',
 '001751.jpg',
 '001754.jpg',
 '001762.jpg',
 '001767.jpg',
 '001769.jpg',
 '001775.jpg',
 '001777.jpg',
 '001780.jpg',
 '001781.jpg',
 '001783.jpg',
 '001787.jpg',
 '001789.jpg',
 '001790.j

In [44]:
len(unique_images)

662

In [23]:
# evaluate
from pycocotools.coco import COCO


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/py37/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [40]:
annFile = 'test.json'
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
